Script de generación de datos documentales TFM Daniel Herranz Segundo

In [17]:
import pandas as pd
import numpy as np
import os
import json
import random

In [18]:
#Variables de los ficheros de datos salida
CurrentAccountCollection_file_out = '../MockData/MongoDB/CurrentAccountCollection/CurrentAccountCollection.json'
PositionKeepingCollection_file_out = '../MockData/MongoDB/PositionKeepingCollection/PositionKeepingCollection.json'
CustomerProfileCollection_file_out = '../MockData/MongoDB/CustomerProfileCollection/CustomerProfileCollection.json'

CurrentAccountCollection_sample_out = '../MockData/MongoDB/CurrentAccountCollection/CurrentAccountCollection_sample.json'
PositionKeepingCollection_sample_out = '../MockData/MongoDB/PositionKeepingCollection/PositionKeepingCollection_sample.json'
CustomerProfileCollection_sample_out = '../MockData/MongoDB/CustomerProfileCollection/CustomerProfileCollection_sample.json'

CurrentAccountCollection_sample_10K_out = '../MockData/MongoDB/CurrentAccountCollection/CurrentAccountCollection_10K_sample.json'
PositionKeepingCollection_sample_10K_out = '../MockData/MongoDB/PositionKeepingCollection/PositionKeepingCollection_10K_sample.json'
CustomerProfileCollection_sample_10K_out = '../MockData/MongoDB/CustomerProfileCollection/CustomerProfileCollection_10K_sample.json'

CurrentAccountCollection_sample_100K_out = '../MockData/MongoDB/CurrentAccountCollection/CurrentAccountCollection_100K_sample.json'
PositionKeepingCollection_sample_100K_out = '../MockData/MongoDB/PositionKeepingCollection/PositionKeepingCollection_100K_sample.json'
CustomerProfileCollection_sample_100K_out = '../MockData/MongoDB/CustomerProfileCollection/CustomerProfileCollection_100K_sample.json'

CurrentAccountCollection_sample_500K_out = '../MockData/MongoDB/CurrentAccountCollection/CurrentAccountCollection_500K_sample.json'
PositionKeepingCollection_sample_500K_out = '../MockData/MongoDB/PositionKeepingCollection/PositionKeepingCollection_500K_sample.json'
CustomerProfileCollection_sample_500K_out = '../MockData/MongoDB/CustomerProfileCollection/CustomerProfileCollection_500K_sample.json'

In [19]:
#Función para resetear ficheros de salida
def reset_files(file):
    if os.path.exists(file):
        os.remove(file)
        print("The file", file ,"have been removed")
    else:
        print("The file", file ,"does not exist")

In [20]:
#Limpiamos los ficheros de salida
reset_files(CurrentAccountCollection_file_out)
reset_files(PositionKeepingCollection_file_out)
reset_files(CustomerProfileCollection_file_out)
reset_files(CurrentAccountCollection_sample_out)
reset_files(PositionKeepingCollection_sample_out)
reset_files(CustomerProfileCollection_sample_out)

The file ../MockData/MongoDB/CurrentAccountCollection/CurrentAccountCollection.json does not exist
The file ../MockData/MongoDB/PositionKeepingCollection/PositionKeepingCollection.json does not exist
The file ../MockData/MongoDB/CustomerProfileCollection/CustomerProfileCollection.json does not exist
The file ../MockData/MongoDB/CurrentAccountCollection/CurrentAccountCollection_sample.json does not exist
The file ../MockData/MongoDB/PositionKeepingCollection/PositionKeepingCollection_sample.json does not exist
The file ../MockData/MongoDB/CustomerProfileCollection/CustomerProfileCollection_sample.json does not exist


# Generación coleccion CustomerProfile para MongoDB

In [21]:
#Carga de la información de dataframes por entidades

Address_df = pd.read_csv('../MockData/Address_1M.csv')
CustomerProfile_df = pd.read_csv('../MockData/CustomerProfile_1M.csv')
Country_df = pd.read_csv('../MockData/base/Country.csv')

In [24]:
print(Address_df.columns)
print(CustomerProfile_df.columns)
print(Country_df.columns)

Index(['PartyId', 'AddressType', 'AddressLine', 'StreetName', 'BuildingNumber',
       'PostCode', 'TownName', 'CountrySubDivision', 'CountryId'],
      dtype='object')
Index(['PartyId', 'PartyNumber', 'PartyType', 'Name', 'FullLegalName',
       'LegalStructure', 'BeneficialOwnership', 'AccountRole', 'EmailAddress',
       'Phone'],
      dtype='object')
Index(['CountryId', 'ShortName', 'Description', 'Code', 'Unnamed: 4'], dtype='object')


In [23]:
#Comentar si modo real
CustomerProfile_df = CustomerProfile_df.sample(1000) 
Address_df = Address_df.sample(1000)

In [25]:
#Indexado PartyId para mejor legibilidad
del(Country_df['CountryId'])
del(Country_df['Unnamed: 4'])

del(Address_df['PartyId'])
del(Address_df['CountryId'])

In [26]:
#Agregamos una columna de pais a las direcciones
CountryColumn = []

for i in range (0,len(Address_df)): 
    CountryArrayAsJSON = json.loads(Country_df.sample(1).to_json(orient ='records'))
    CountryColumn.append(CountryArrayAsJSON)    

Address_df['Country'] = CountryColumn

#Agreación de direcciones (Address) a la colección Customer Profile
#Tratamiento de las direcciones. Creamos de forma aleatoria entre 1 y 10 como agregado del CustomerProfile
AddressColumn = []

for i in range (0,len(CustomerProfile_df)): 
    AddressArrayAsJSON = json.loads(Address_df.sample(random.choice(range(1,3))).to_json(orient ='records'))
    AddressColumn.append(AddressArrayAsJSON)    
    
CustomerProfile_df['Address'] = AddressColumn
CustomerProfile_df.sample(2)

,PartyId,PartyNumber,PartyType,Name,FullLegalName,LegalStructure,BeneficialOwnership,AccountRole,EmailAddress,Phone,Address
563410,4370,4370,sol,Walter,Mr. Winston Ullrich PhD,Mr.,0,UK.OBIE.Principal,koepp.andres@example.com,05188766010,"[{'AddressType': 'New', 'AddressLine': '192 Co..."
112766,3414,3414,sol,Robb,Aidan Ledner,Prof.,1,UK.OBIE.Principal,garret81@example.net,1-944-595-77,"[{'AddressType': 'Nort', 'AddressLine': '89643..."


In [28]:
#Volcado fichero JSON
CustomerProfile_df.to_json(CustomerProfileCollection_sample_out,orient ='records') #Sample for testing

#CustomerProfile_df.to_json(CustomerProfileCollection_file_out,orient ='records') #Final file


# Generación colección CurrentAccount para MongoDB

In [34]:
#Carga de datasets necesarias para la colección

AccountInfo_df = pd.read_csv('../MockData/AccountInfo_1M.csv')
CurrentAccount_df = pd.read_csv('../MockData/CurrentAccount_1M.csv')

In [35]:
AccountInfo_df.columns

Index(['AccountInfoId', 'AccountId', 'SchemeName', 'Identification', 'Name'], dtype='object')

In [36]:
CurrentAccount_df.columns

Index(['AccountId', 'PartyId', 'Status', 'StatusUpdateDateTime', 'AccountType',
       'NickName', 'OpeningDate', 'AccountSubType'],
      dtype='object')

In [37]:
#Limpieza de columnas residuales y no necesarias en el documento
del(AccountInfo_df['AccountInfoId'])
del(AccountInfo_df['AccountId'])

In [38]:
CurrentAccount_df.sample(1)

,AccountId,PartyId,Status,StatusUpdateDateTime,AccountType,NickName,OpeningDate,AccountSubType
276322,2502,2623,Enabled,1995-10-18 05:02:59.00,Business,unleash B2B experiences,1977-01-04 11:03:09.00,SAI


In [39]:
AccountInfo_df.sample(1)

,SchemeName,Identification,Name
876648,UK.OBIE.SortCodeAccountNumber,5.0,Blick-Stoltenberg


In [40]:
#Creamos un sample. Testing (comentar en generación real)
CurrentAccount_df = CurrentAccount_df.sample(10000)
AccountInfo_df = AccountInfo_df.sample(10000)

In [41]:
#Generamos la colección agregando AccountInfo a las cuentas

AccountInfoColumn = []
CurrentAccountCollection = pd.DataFrame()

len(CurrentAccount_df)

for i in range (0,len(CurrentAccount_df)):     
    AccountInfoArrayAsJSON = json.loads(AccountInfo_df.sample(random.choice(range(1,3))).to_json(orient ='records'))
    AccountInfoColumn.append(AccountInfoArrayAsJSON)    
    
CurrentAccount_df['AccountInfo'] = AccountInfoColumn


In [42]:
CurrentAccount_df.sample(1)

,AccountId,PartyId,Status,StatusUpdateDateTime,AccountType,NickName,OpeningDate,AccountSubType,AccountInfo
1018154,7205,7380,Enabled,1999-12-02 17:20:33.00,Particular,embrace vertical users,1987-10-05 11:56:38.00,SAI,[{'SchemeName': 'ES.OBIE.SortCodeAccountNumber...


In [43]:
len(CurrentAccount_df)

10000

In [45]:
#Volcado fichero JSON
CurrentAccount_df.to_json(CurrentAccountCollection_sample_out,orient ='records') #Sample for testing

#CurrentAccount_df.to_json(CurrentAccountCollection_file_out,orient ='records')

# Generación colección PositionKeeping para MongoDB

In [46]:
#Carga de datasets necesarias para la colección
Amount_df = pd.read_csv('../MockData/Amount_1M.csv')
CreditLine_df = pd.read_csv('../MockData/CreditLine_1M.csv')
Currency_df = pd.read_csv('../MockData/base/Currency.csv')
PositionKeeping_df = pd.read_csv('../MockData/Position_Keeping_1M.csv')

In [47]:
#Verificación de columnas
print(Amount_df.columns)
print(Country_df.columns)
print(CreditLine_df.columns)
print(Currency_df.columns)
print(PositionKeeping_df.columns)

Index(['AmountId', 'CurrencyId', 'Amount'], dtype='object')
Index(['ShortName', 'Description', 'Code'], dtype='object')
Index(['CreditLineId', 'CurrencyId', 'Amount', 'Type', 'Included'], dtype='object')
Index(['CurrencyId', 'Code', 'Description', 'Unnamed: 3'], dtype='object')
Index(['AccountId', 'DateTime', 'CreditDebitIndicator', 'Type', 'AmountId',
       'CreditLineId'],
      dtype='object')


In [48]:
#Limpieza de columnas residuales y no necesarias en el documento
del(Amount_df['CurrencyId'])
del(Amount_df['AmountId'])

del(CreditLine_df['CreditLineId'])
del(CreditLine_df['CurrencyId'])

del(Currency_df['CurrencyId'])

del(PositionKeeping_df['CreditLineId'])
del(PositionKeeping_df['AmountId'])
del(PositionKeeping_df['AccountId'])

In [49]:
#Creamos samples. Testing (comentar en generación real)
Amount_df = Amount_df.sample(10000)
CreditLine_df = CreditLine_df.sample(10000)
PositionKeeping_df = PositionKeeping_df.sample(10000)

In [50]:
#Generamos la colección PositionKeeping

AmountColumn = []
CreditLineColumn = []

for i in range (0,len(PositionKeeping_df)):    
    valueAmount = {'Currency': Currency_df.sample(1).to_json(orient = 'records'), 'Amount' : Amount_df.sample(1).to_json(orient ='records')}
    AmountColumn.append(valueAmount)
    valueCreditLine = {
    'Included' : CreditLine_df.sample(1)['Included'].to_json(orient ='records'),                    
    'Type': CreditLine_df.sample(1)['Type'].to_json(orient ='records'),
    'Amount': { 'Currency': Currency_df.sample(1).to_json(orient = 'records'), 'Amount' : CreditLine_df.sample(1).to_json(orient ='records') }
    }
    CreditLineColumn.append(valueCreditLine)


PositionKeeping_df['Amount'] = AmountColumn
PositionKeeping_df['CreditLine'] = CreditLineColumn
PositionKeeping_df.sample(1)

,DateTime,CreditDebitIndicator,Type,Amount,CreditLine
704999,1983-11-21 14:27:35.00,Credit,Business,"{'Currency': '[{""Code"":""IND"",""Description"":""Es...","{'Included': '[0]', 'Type': '[null]', 'Amount'..."


In [51]:
len(PositionKeeping_df)

10000

In [52]:
#Volcado fichero JSON
PositionKeeping_df.to_json(PositionKeepingCollection_sample_out,orient ='records') #Final file

#PositionKeeping_df.to_json(PositionKeepingCollection_file_out,orient ='records') #Final file
